In [1]:
point_A = [55.75222, 37.61556]  # Moscow
point_B = [50.45466, 30.5238]   # Kyiv

num_waypoints = 5
num_paths = 3

In [2]:
from terrain import calculate_distance

straight_line_distance:float = calculate_distance(
    point_A[0],
    point_A[1],
    point_B[0],
    point_B[1]
)

print(f'Straight line distance: {straight_line_distance} km')

Straight line distance: 756.2941363684055 km


In [3]:
import random
from geopy.distance import great_circle

def generate_random_waypoints(start:tuple[float, float], end:tuple[float, float], num_waypoints:int) -> list[tuple[int,int]]:
    """
    Generates random waypoints between a start and endpoint.
    
    Args: 
        start (tuple[float, float]): starting latitude and longitude.
        end (tuple[float, float]): ending latitude and longitude.
        num_waypoints (int) number of random waypoints to generate.
    
    Returns 
        list[tuple[float, float]]: list of tuples containing the waypoints.
    """
    waypoints:list[tuple[float, float]] = [start]
    
    # Generate random waypoints
    for _ in range(num_waypoints):
        # Randomly choose a fraction between 0 and 1 
        fraction = random.uniform(0, 1)

        # Add the fraction to the lat/long of the start and end to generate a random point between the two
        lat:float = start[0] + fraction * (end[0] - start[0])
        lon:float = start[1] + fraction * (end[1] - start[1])
        waypoints.append((lat, lon))
    
    # Add the end location to the waypoints so we end in the right place 
    waypoints.append(end)

    # Return the populated list 
    return waypoints


def create_random_paths(start:tuple[float, float], end:tuple[float, float], num_waypoints:int, num_paths:int) -> list[list[tuple[float, float]]]:
    """
    Creates multiple random paths between a start and endpoint.
    
    Args: 
        start (tuple[float, float]): starting latitude and longitude.
        end (tuple[float, float]): ending latitude and longitude.
        num_waypoints (int): number of random waypoints to generate.
        num_paths (int): number of paths to generate

    Returns: 
        list[list[tuple[float, float]]]: list of lists containing paths, where each path is a list of tuples (lat, lon).
    """
    # Init empty list of paths
    paths = []
    
    # Generate num_paths possible paths 
    for _ in range(num_paths):
        waypoints = generate_random_waypoints(start, end, num_waypoints)
        paths.append(waypoints)
    
    return paths

In [4]:
random_paths = create_random_paths(point_A, point_B, num_waypoints, num_paths)

# Print generated paths
for i, path in enumerate(random_paths):
    print(f"Path {i + 1}:")
    for waypoint in path:
        print(f"  {waypoint}")
    print()

Path 1:
  [55.75222, 37.61556]
  (52.465020549671266, 33.2150379532722)
  (54.450910863761514, 35.873517988203886)
  (54.15187197660851, 35.473199347479444)
  (53.986827074067094, 35.25225634012377)
  (50.908860623559605, 31.131831209487967)
  [50.45466, 30.5238]

Path 2:
  [55.75222, 37.61556]
  (53.30587162563184, 34.34067202376016)
  (53.054281636966564, 34.00387247490807)
  (52.036974911372575, 32.64202000994337)
  (54.372250801965805, 35.76821700438485)
  (52.8814980273803, 33.77256978251394)
  [50.45466, 30.5238]

Path 3:
  [55.75222, 37.61556]
  (53.15364181557452, 34.13688437854763)
  (52.15147931456199, 32.79530524812142)
  (54.42396097524736, 35.837440597599695)
  (50.97525816161543, 31.220716546224654)
  (52.76756988656275, 33.62005597768223)
  [50.45466, 30.5238]

